#### import functions

In [9]:
%run functions_NoiseStudy.py
%matplotlib 

# global variables for plot savings
fs = 13
type_plot = ['png', 'svg']

Using matplotlib backend: MacOSX


### 0. Directory for data storage

In [2]:
# actual evaluation time
now = datetime.datetime.now()
today = now.strftime("%Y%m%d")

# output folder
save_dir_plots = 'plots/' + today + '_measurement'
if not os.path.exists(save_dir_plots):
    os.makedirs(save_dir_plots)
    
save_dir_res = 'Results/' + today + '_measurement'
if not os.path.exists(save_dir_res):
    os.makedirs(save_dir_res)

### 1. Load data

In [3]:
# [USER INPUT]
# calibration data 
file_calib = 'Results/20210531_calibration/20210531-1455_calibration_1RoI.hdf5'

# measurement data - images of the optode
file_meas = '/Volumes/HIS-CAMEL/04measurementData/20201127_Noise-vs-resolution-paper/Klaus_Optode_noise_study_26-11-2020/O2gradient_experiement/'


##### additional information

In [4]:
# image cropping to region of interest (RoI) that should be analyzed
RoI_op = [[(730, 200), (730, 1250), (1290, 1250), (1290, 200)],   # optode1
          [(1560, 200), (1560, 1250), (2100, 1250), (2100, 200)]] # optode2

# Image resolution - determine conversion factor px -> mm - user definition using
# f.ex. imageJ
px2mm, dpi = image_resolution(px=840.6646, dist_mm=30., inch=1.1811)
print('Image resolution: 1cm equals: {:.2f}px, i.e. {:.0f}dpi'.format(px2mm, dpi))

Image resolution: 1cm equals: 28.02px, i.e. 712dpi


In [5]:
# load calibration data
dic_header, dinitial, dnorm, dpara = load_calibResults(path_calib=file_calib)

# load measurement data and crop according to RoI
dict_ratio_run1, dict_ratio_run2 = splitImage(path=file_meas, RoI_op=RoI_op)

### 2. Determine O2 concentration in each pixel and each measurement
using the Stern-Volmer equation and the respective calibration fit

In [19]:
# one example of one optode
inp = input('Which optode and which setting shall be visualized? ') # optode1, set2
run = input('Which measurement shall be visualized - 1 or 2?') # 1,2

Which optode and which setting shall be visualized? optode1, set2
Which measurement shall be visualized - 1 or 2?2


In [13]:
# [USER input] - define surface depth within the image
surface = (12.9, 12.) # optode 1, optode2

# Oxygen concentration determination
dO2, dO2_av, dO2_SD = o2_calculation(inp=inp, dict_ratio_run1=dict_ratio_run1, run=run, 
                                     dpara=dpara, surface=surface, px2mm=px2mm, 
                                     splitdata=True, dict_ratio_run2=dict_ratio_run2)

optode1 ...
optode2 ...
post-processing...


### 3. Plot optode as 3D or 2D

In [17]:
# 3D visualization
sns.set_style('darkgrid')
fig3D, ax3D = plot_wholeImage3D(dO2_mean=dO2_av[inp.split(',')[0]][inp.split(',')[1].strip()],
                                unit='mm', pad=2)
plt.show()   

In [18]:
# 2D projection
depth_range = (-2.5, 2.5)
width_range = (2., 20.)

sns.set_style('ticks')
fig2D, ax2D = plot_optode2D(o=inp.split(',')[0], s=inp.split(',')[1].strip(), px2mm=px2mm, 
                            surface=surface, dO2_av=dO2_av, depth_range=depth_range,
                            width_range=width_range, vmin=0, vmax=100)
plt.show() 

### 4. Save results
save figures

In [51]:
now = datetime.datetime.now() 
name3D = save_dir_plots + '/' + now.strftime("%Y%m%d-%H%M%S") + '_optode3D_' + '-'.join([i.strip() for i in inp.split(',')])
name2D = save_dir_plots + '/' + now.strftime("%Y%m%d-%H%M%S") + '_optode2D_' + '-'.join([i.strip() for i in inp.split(',')])
for i in type_plot:
    fig3D.savefig(name3D + '.{}'.format(i), dpi=dpi) 
    fig2D.savefig(name2D + '.{}'.format(i), dpi=dpi) 

save determined O2

In [52]:
eval_time = now.strftime("%Y%m%d-%H%M")
save_name = save_dir_res + '/' + eval_time + '_O2calc_run' + run + '.hdf5'
save_o2results(save_name=save_name, inp=inp, file_meas=file_meas, RoI_op=RoI_op,
               px2mm=px2mm, surface=surface, dO2_av=dO2_av, dO2_SD=dO2_SD)

saving done
